In [ ]:
# 隨機十篇id
# M.1127219109.A.837
# M.1127227032.A.F3B
# M.1127365129.A.D61
# M.1127395683.A.DB9
# M.1127526873.A.CD5
# M.1127533364.A.C32
# M.1127544042.A.E06
# M.1127586620.A.EDB
# M.1127588070.A.525
# M.1127590679.A.006


In [19]:
# 用比較快快的json parser : ujson
try:
    import ujson as json
except ImportError:
    try:
        import simplejson as json
    except ImportError:
        import json

import math


# load tfidf
def readJson2Data(filename):
  f = open(filename, "r", encoding='UTF-8')
  docText = f.read()
  f.close()
  return json.loads(docText)




In [2]:
SAMPLE = 'sample1000.json'
ALLDATA = 'gossiping_dataset_doc_vector.json'
data_tfidf = readJson2Data(SAMPLE)



In [3]:
# tools
def saveSmallData(data):
    jsObj = json.dumps(data)
    
    fileObject = open('sample1000.json', 'w')
    fileObject.write(jsObj)
    fileObject.close()  
# saveSmallData(sample)    

In [9]:


sample = {}
counter = 0
for k in data_tfidf.keys():
    if counter == 10:
        break
    print(k)
    data_tfidf[k]
    sample[k] = data_tfidf[k]

    counter = counter + 1

M.1127219109.A.837
M.1127227032.A.F3B
M.1127365129.A.D61
M.1127395683.A.DB9
M.1127526873.A.CD5
M.1127533364.A.C32
M.1127544042.A.E06
M.1127586620.A.EDB
M.1127588070.A.525
M.1127590679.A.006


In [18]:
counter = 0

for i in data_tfidf['M.1127219109.A.837'].keys():
    print(data_tfidf['M.1127219109.A.837'][i])
counter = counter + 1


0.061299164513706546
0.06357538203009586
0.025219464223066857
0.032591316967399454
0.021506926704879775
0.021856437389587275
0.14448621613593726
0.15319949509168582
0.03547108966373669
0.056285334040362754
0.07067876523062343
0.031794190785616086
0.04574606352511516
0.08215367591688302
0.04777362743650533
0.0438859895048074
0.10056986904833362
0.11695759348628047
0.06646007985172464
0.01533018125818423
0.012884607948533421
0.12597902138229752
0.04653726094947778
0.058187733715991063
0.0198541810762566
0.09532743998015608
0.037878769526195226
0.05697700366876267
0.058224117245979874
0.05781192595671208
0.02612598456066965
0.03863713286884628
0.1478508394755249
0.036035144068431675
0.039715581416385856
0.12883256787972408
0.061138084808307375
0.04786534868915153
0.09827970654885833
0.14141834621135066
0.07836175128801515
0.06386091620198975
0.08133883752273477
0.08384177220516788
0.03357076680787673
0.061972395652619944
0.05316728885964829
0.06663442572791275
0.12775479168331025
0.230324

In [5]:
def askTargetUser(uid):
    name = input("請輸入需要查詢的使用者名稱")
    return name


In [20]:
def calNormalizedLength(ida):
    vector = {}
    sum = 0
    for word in data_tfidf[ida].keys():
        sum = sum + math.pow(data_tfidf[ida][word], 2)
    
    sqrt = math.sqrt(sum)

    # 每項除除平方根
    for word in data_tfidf[ida].keys():
        vector[word] = data_tfidf[ida][word] / sqrt
    return vector


In [22]:
def findCommonWordSet(ida,idb):
    CommonWordSet = set()
    ASet = set( data_tfidf[ida].keys() )
    Bset = set( data_tfidf[idb].keys() )

    for token in Aset:
        if token in Bset:
            CommonWordSet.append(token)
    return CommonWordSet

In [ ]:
def cos(ida,idb):
    # calNormalizedLength
    ida_NL = calNormalizedLength(ida)
    idb_NL = calNormalizedLength(idb)

    # findCommonWordSet
    commonWordSet = findCommonWordSet(ida,idb)

    # calCos
    